In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres
pd.set_option('display.max_columns', None)
import psycopg2

In [2]:
df_partidos = pd.read_csv("data/Team_Stats_Normalizado.csv")
df_completo = pd.read_csv("data/jugadores_info_normalizado.csv")
df_clubs = df_completo[["club", "clubID"]].drop_duplicates().dropna().reset_index().drop(columns = ["index"])

In [3]:
df_partidos.drop(columns = ["home_team", "away_team"], inplace = True)

In [4]:
diccionario_clubs = dict(zip(df_clubs["club"], df_clubs["clubID"]))

In [5]:
df_partidos["id_club_local"] = df_partidos["Local"].map(diccionario_clubs)
df_partidos["id_club_visitante"] = df_partidos["Visitante"].map(diccionario_clubs)
df_partidos["id_club_stats"] = df_partidos["Stats Equipo"].map(diccionario_clubs)

In [6]:
df_partidos.rename(columns = {'Seleccionar esquema': 'Esquema'}, inplace = True)

In [7]:
df_partidos.columns = df_partidos.columns.str.replace(' ', '_')

In [8]:
df_prueba = df_partidos.copy()

In [9]:
# Creamos una clave única por partido
df_prueba['match_id'] = df_prueba['match_id'].astype(int)  # asegúrate de que sea tipo entero si no lo está
# Separar columnas fijas (que no queremos duplicar) de métricas
columnas_clave = ['Fecha', 'Partido', 'Resultado', 'match_id', 'Local', 'Visitante','Competición', 'Duración', 'id_club_local',  'id_club_visitante', 'id_club_stats']
columnas_metricas = df_prueba.columns.difference(columnas_clave).tolist()

# Pivotar el DataFrame para tener columnas por equipo
df_local = df_prueba[df_partidos['Local'] == df_prueba['Stats_Equipo']].copy()
df_visitante = df_prueba[df_partidos['Visitante'] == df_prueba['Stats_Equipo']].copy()

# Renombramos columnas
df_local = df_local.set_index('match_id')
df_local.columns = [col + '_local' if col not in columnas_clave else col for col in df_local.columns]

df_visitante = df_visitante.set_index('match_id')
df_visitante.columns = [col + '_visitante' if col not in columnas_clave else col for col in df_visitante.columns]

# Unimos los dos DataFrames
df_final = df_local.join(df_visitante,lsuffix='_L', rsuffix='_V', how='inner').reset_index()

In [10]:
df_final.drop(columns=([ 'Stats_Equipo_local',
 'id_club_stats_L',
 'Fecha_V',
 'Partido_V',
 'Resultado_V',
 'Local_V',
 'Visitante_V',
 'Competición_V',
 'Duración_V',
 'Tiros_libres_con_remate.1_local',
 'Tiros_libres_con_remate.1_visitante',
 'Stats_Equipo_visitante',
 'id_club_stats_V',
 'id_club_local_V',
 'id_club_visitante_V',
 'Jornada_local']), inplace=True)

In [11]:
df_final.rename(columns={'Fecha_L':'Fecha',
 'Partido_L':'Partido',
 'Resultado_L': 'Resultado',
 'Local_L':'Local',
 'Visitante_L':'Visitante',
 'Competición_L':'Competición',
 'Duración_L':'Duración',
 'Jornada_visitante':'Jornada',
 'id_club_local_L' :'id_club_local',
 'id_club_visitante_L' :'id_club_visitante',
}, inplace=True)

In [12]:
df_final.columns = df_final.columns.str.replace('%', 'porcentaje')
df_final.columns = df_final.columns.str.replace(',', '')

In [13]:
len(df_final.columns.tolist())

193

In [14]:
df_final["Jornada"]

0       1
1       1
2       1
3       1
4       1
       ..
375    38
376    38
377    38
378    38
379    38
Name: Jornada, Length: 380, dtype: int64

In [18]:
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_final,"partidos", cur, conn)

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)


In [19]:
df_final.to_csv("data/Tabla_Partidos.csv", index=False)